In [1]:
from transit_parser import *

In [2]:
TEST = './test/'
TRAINS = './trains/'
def examine_train(filename, folder=TEST):
    train_name = folder + filename
    t = TrainParser(train_name)
    times = t.get_stop_times()
    rows = t.get_rows(times)
    df = t.get_df(rows)
    performance = t.join_schedule(df)
    return performance[['from', 'to','status', 'time', 'expected','train_id', 'line']]

In [3]:
pd.set_option('display.height', 100)

height has been deprecated.



## NJ Transit 

In [7]:
examine_train('2018_02_02_3714')

[786.0, 42414.0, 85614.0]


,from,to,status,time,expected,train_id
stop_num,,,,,,
1,Jersey Avenue,Jersey Avenue,Departed,2018-02-02 07:55:10,2018-02-02 07:55:00,3714
2,Jersey Avenue,New Brunswick,Departed,2018-02-02 08:02:11,2018-02-02 08:01:00,3714
3,New Brunswick,Edison,Departed,2018-02-02 08:07:13,2018-02-02 08:06:00,3714
4,Edison,Metuchen,Departed,2018-02-02 08:13:08,2018-02-02 08:12:00,3714
5,Metuchen,Metropark,Departed,2018-02-02 08:18:07,2018-02-02 08:17:00,3714
6,Metropark,Newark Penn Station,Departed,2018-02-02 08:35:07,2018-02-02 08:35:00,3714
7,Newark Penn Station,Secaucus Upper Lvl,Departed,2018-02-02 08:45:08,2018-02-02 08:43:00,3714
8,Secaucus Upper Lvl,New York Penn Station,Departed,2018-02-02 08:58:00,2018-02-02 08:57:00,3714


In [13]:
examine_train('2018_02_02_3928')

[966.0, 42234.0, 85434.0]


,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Trenton,Trenton,Departed,2018-02-02 07:49:01,2018-02-02 07:50:00,3928,Northeast Corrdr
2,Trenton,Hamilton,Departed,2018-02-02 07:57:01,2018-02-02 07:57:00,3928,Northeast Corrdr
3,Hamilton,Princeton Junction,Departed,2018-02-02 08:05:03,2018-02-02 08:05:00,3928,Northeast Corrdr
4,Princeton Junction,New Brunswick,Departed,2018-02-02 08:21:06,2018-02-02 08:21:00,3928,Northeast Corrdr
5,New Brunswick,Metropark,Departed,2018-02-02 08:31:07,2018-02-02 08:31:00,3928,Northeast Corrdr
6,Metropark,Newark Airport,Departed,2018-02-02 08:45:05,2018-02-02 08:44:00,3928,Northeast Corrdr
7,Newark Airport,Newark Penn Station,Departed,2018-02-02 08:53:03,2018-02-02 08:51:00,3928,Northeast Corrdr
8,Newark Penn Station,Secaucus Upper Lvl,Departed,2018-02-02 09:03:09,2018-02-02 09:00:00,3928,Northeast Corrdr
9,Secaucus Upper Lvl,New York Penn Station,Departed,2018-02-02 09:21:00,2018-02-02 09:14:00,3928,Northeast Corrdr


In [14]:
examine_train('2018_02_02_3712')

[729.0, 42471.0, 85671.0]


,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Jersey Avenue,Jersey Avenue,Departed,2018-02-02 07:39:14,2018-02-02 07:39:00,3712,Northeast Corrdr
2,Jersey Avenue,New Brunswick,Departed,2018-02-02 07:47:11,2018-02-02 07:44:00,3712,Northeast Corrdr
3,New Brunswick,Edison,Departed,2018-02-02 07:51:10,2018-02-02 07:49:00,3712,Northeast Corrdr
4,Edison,Metuchen,Departed,2018-02-02 07:55:10,2018-02-02 07:55:00,3712,Northeast Corrdr
5,Metuchen,Metropark,Departed,2018-02-02 08:00:13,2018-02-02 08:00:00,3712,Northeast Corrdr
6,Metropark,Newark Penn Station,Departed,2018-02-02 08:16:13,2018-02-02 08:16:00,3712,Northeast Corrdr
7,Newark Penn Station,Secaucus Upper Lvl,Departed,2018-02-02 08:26:06,2018-02-02 08:23:00,3712,Northeast Corrdr
8,Secaucus Upper Lvl,New York Penn Station,Departed,2018-02-02 08:35:00,2018-02-02 08:36:00,3712,Northeast Corrdr


In [7]:
examine_train('2018_01_29_0067')

[43127.0, 73.0, 43273.0]
[42467.0, 733.0, 43933.0]
[41207.0, 1993.0, 45193.0]
[41152.0, 2048.0, 45248.0]


,from,to,status,time,expected,train_id
stop_num,,,,,,
1,Hoboken,Hoboken,Departed,2018-01-29 22:01:00,2018-01-29 21:58:00,0067
2,Hoboken,Secaucus Lower Lvl,Departed,2018-01-29 22:08:58,2018-01-29 22:08:00,0067
3,Secaucus Lower Lvl,Kingsland,Departed,2018-01-29 22:15:53,2018-01-29 22:14:00,0067
4,Kingsland,Lyndhurst,Departed,2018-01-29 22:17:52,2018-01-29 22:17:00,0067
5,Lyndhurst,Delawanna,Departed,2018-01-29 22:20:54,2018-01-29 22:20:00,0067
6,Delawanna,Passaic,Departed,2018-01-29 22:23:50,2018-01-29 22:23:00,0067
7,Passaic,Clifton,Departed,2018-01-29 22:26:56,2018-01-29 22:26:00,0067
8,Clifton,Paterson,Departed,2018-01-29 22:33:57,2018-01-29 22:31:00,0067
9,Paterson,Hawthorne,Departed,2018-01-29 22:37:57,2018-01-29 22:35:00,0067


In [19]:
examine_train('2018_01_31_3714')

,from,to,status,time,expected,train_id
stop_num,,,,,,
1,Jersey Avenue,Jersey Avenue,Departed,2018-01-31 07:55:14,2018-01-31 07:55:00,3714
2,Jersey Avenue,New Brunswick,Departed,2018-01-31 08:02:05,2018-01-31 08:01:00,3714
3,New Brunswick,Edison,Departed,2018-01-31 08:07:06,2018-01-31 08:06:00,3714
4,Edison,Metuchen,Departed,2018-01-31 08:12:11,2018-01-31 08:12:00,3714
5,Metuchen,Metropark,Departed,2018-01-31 08:17:05,2018-01-31 08:17:00,3714
6,Metropark,Newark Penn Station,Departed,2018-01-31 09:06:06,2018-01-31 08:35:00,3714
7,Newark Penn Station,Secaucus Upper Lvl,Departed,2018-01-31 09:16:04,2018-01-31 08:43:00,3714
8,Secaucus Upper Lvl,New York Penn Station,Departed,2018-01-31 09:24:00,2018-01-31 08:57:00,3714


In [4]:
examine_train('2018_02_02_0046')


,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Port Jervis,Port Jervis,Departed,2018-02-02 05:02:03,2018-02-02 05:03:00,0046,Bergen Co. Line
2,Port Jervis,Otisville,Departed,2018-02-02 05:25:08,2018-02-02 05:21:00,0046,Bergen Co. Line
3,Otisville,Middletown NY,Departed,2018-02-02 05:36:08,2018-02-02 05:32:00,0046,Bergen Co. Line
4,Middletown NY,Campbell Hall,Departed,2018-02-02 05:44:07,2018-02-02 05:40:00,0046,Bergen Co. Line
5,Campbell Hall,Salisbury Mills-Cornwall,Departed,2018-02-02 05:53:13,2018-02-02 05:52:00,0046,Bergen Co. Line
6,Salisbury Mills-Cornwall,Harriman,Departed,2018-02-02 06:08:07,2018-02-02 06:06:00,0046,Bergen Co. Line
7,Harriman,Tuxedo,Departed,2018-02-02 06:20:11,2018-02-02 06:16:00,0046,Bergen Co. Line
8,Tuxedo,Sloatsburg,Departed,2018-02-02 06:23:09,2018-02-02 06:21:00,0046,Bergen Co. Line
9,Sloatsburg,Suffern,Departed,2018-02-02 06:30:17,2018-02-02 06:28:00,0046,Bergen Co. Line


In [8]:
examine_train('2018_02_01_3806')


,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Trenton,Trenton,Departed,2018-02-02 03:49:28,2018-02-01 03:48:00,3806,Northeast Corrdr
2,Trenton,Hamilton,Departed,2018-02-02 04:02:27,2018-02-01 03:54:00,3806,Northeast Corrdr
3,Hamilton,Princeton Junction,Departed,2018-02-02 04:05:28,2018-02-01 04:00:00,3806,Northeast Corrdr
4,Princeton Junction,New Brunswick,Departed,2018-02-02 04:18:32,2018-02-01 04:14:00,3806,Northeast Corrdr
5,New Brunswick,Edison,Departed,2018-02-02 04:22:26,2018-02-01 04:18:00,3806,Northeast Corrdr
6,Edison,Metuchen,Departed,2018-02-02 04:26:31,2018-02-01 04:23:00,3806,Northeast Corrdr
7,Metuchen,Metropark,Departed,2018-02-02 04:31:29,2018-02-01 04:27:00,3806,Northeast Corrdr
8,Metropark,Rahway,Departed,2018-02-02 04:37:30,2018-02-01 04:34:00,3806,Northeast Corrdr
9,Rahway,Linden,Departed,2018-02-02 04:41:38,2018-02-01 04:38:00,3806,Northeast Corrdr


In [6]:
examine_train('2018_02_02_3713', TRAINS)


DEPARTED New York Penn Station DEPARTED
DEPARTED Secaucus Upper Lvl DEPARTED
DEPARTED Newark Penn Station DEPARTED
DEPARTED Newark Airport DEPARTED
DEPARTED North Elizabeth DEPARTED
DEPARTED Elizabeth DEPARTED
DEPARTED Linden DEPARTED
DEPARTED Rahway DEPARTED
DEPARTED Metropark DEPARTED
DEPARTED New Brunswick DEPARTED
DEPARTED Jersey Avenue DEPARTED


,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,New York Penn Station,New York Penn Station,Departed,2018-02-02 07:59:11,2018-02-02 08:00:00,3713,Northeast Corrdr
2,New York Penn Station,Secaucus Upper Lvl,Departed,2018-02-02 08:11:11,2018-02-02 08:10:00,3713,Northeast Corrdr
3,Secaucus Upper Lvl,Newark Penn Station,Departed,2018-02-02 08:20:09,2018-02-02 08:19:00,3713,Northeast Corrdr
4,Newark Penn Station,Newark Airport,Departed,2018-02-02 08:26:06,2018-02-02 08:25:00,3713,Northeast Corrdr
5,Newark Airport,North Elizabeth,Departed,2018-02-02 08:29:11,2018-02-02 08:29:00,3713,Northeast Corrdr
6,North Elizabeth,Elizabeth,Departed,2018-02-02 08:33:05,2018-02-02 08:32:00,3713,Northeast Corrdr
7,Elizabeth,Linden,Departed,2018-02-02 08:39:11,2018-02-02 08:38:00,3713,Northeast Corrdr
8,Linden,Rahway,Departed,2018-02-02 09:02:05,2018-02-02 08:42:00,3713,Northeast Corrdr
9,Rahway,Metropark,Departed,2018-02-02 09:09:08,2018-02-02 08:51:00,3713,Northeast Corrdr


In [4]:
examine_train('2018_02_01_5458', TRAINS)

DEPARTED Raritan DEPARTED
DEPARTED Somerville DEPARTED
DEPARTED Bridgewater DEPARTED
DEPARTED Bound Brook DEPARTED
DEPARTED Netherwood DEPARTED
DEPARTED Netherwood DEPARTED
DEPARTED Netherwood DEPARTED
DEPARTED Fanwood DEPARTED
DEPARTED Westfield DEPARTED
DEPARTED Cranford DEPARTED
DEPARTED Roselle Park DEPARTED
DEPARTED Union DEPARTED
DEPARTED Newark Penn Station DEPARTED


,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Raritan,Raritan,Departed,2018-02-01 23:39:07,2018-02-01 23:38:00,5458,Raritan Valley
2,Raritan,Somerville,Departed,2018-02-01 23:42:03,2018-02-01 23:41:00,5458,Raritan Valley
3,Somerville,Bridgewater,Departed,2018-02-01 23:46:07,2018-02-01 23:45:00,5458,Raritan Valley
4,Bridgewater,Bound Brook,Departed,2018-02-01 23:49:07,2018-02-01 23:48:00,5458,Raritan Valley
5,Bound Brook,Netherwood,Departed,2018-02-02 00:23:05,2018-02-01 23:54:00,5458,Raritan Valley
6,Netherwood,Netherwood,Departed,2018-02-02 00:24:09,2018-02-01 23:59:00,5458,Raritan Valley
7,Netherwood,Netherwood,Departed,2018-02-02 00:25:12,2018-02-01 24:02:00,5458,Raritan Valley
8,Netherwood,Fanwood,Departed,2018-02-02 00:26:06,2018-02-01 24:05:00,5458,Raritan Valley
9,Fanwood,Westfield,Departed,2018-02-02 00:31:06,2018-02-01 24:10:00,5458,Raritan Valley


In [15]:
trip_stops[trip_stops['block_id'] == '5458']

,trip_id,expected,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,trip_headsign,direction_id,block_id,shape_id
52821,4148,23:38:00,23:38:00,129,1,0,0,0.0000,15,8,NEWARK PENN STATION,0,5458,2531
52822,4148,23:41:00,23:41:00,138,2,0,0,1.1678,15,8,NEWARK PENN STATION,0,5458,2531
52823,4148,23:45:00,23:45:00,24,3,0,0,4.5040,15,8,NEWARK PENN STATION,0,5458,2531
52824,4148,23:48:00,23:48:00,21,4,0,0,5.6153,15,8,NEWARK PENN STATION,0,5458,2531
52825,4148,23:54:00,23:54:00,36,5,0,0,9.8570,15,8,NEWARK PENN STATION,0,5458,2531
52826,4148,23:59:00,23:59:00,120,6,0,0,12.7970,15,8,NEWARK PENN STATION,0,5458,2531
52827,4148,24:02:00,24:02:00,102,7,0,0,13.9417,15,8,NEWARK PENN STATION,0,5458,2531
52828,4148,24:05:00,24:05:00,44,8,0,0,15.1930,15,8,NEWARK PENN STATION,0,5458,2531
52829,4148,24:10:00,24:10:00,155,9,0,0,17.2816,15,8,NEWARK PENN STATION,0,5458,2531
52830,4148,24:14:00,24:14:00,32,10,0,0,21.9136,15,8,NEWARK PENN STATION,0,5458,2531


## Amtrak

In [16]:
examine_train('2018_02_02_A644')

,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Philadelphia,Philadelphia,Departed,2018-02-02 09:46:11,None,A644,KEYSTONE
2,Philadelphia,Trenton,Departed,2018-02-02 10:17:06,None,A644,KEYSTONE
3,Trenton,Newark Penn Station,Departed,2018-02-02 10:49:14,None,A644,KEYSTONE
4,Newark Penn Station,New York Penn Station,Departed,2018-02-02 11:06:06,None,A644,KEYSTONE


In [14]:
examine_train('2018_01_30_A643')

,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Newark Penn Station,Newark Penn Station,Departed,2018-01-30 09:51:18,None,A643,KEYSTONE
2,Newark Penn Station,Trenton,Departed,2018-01-30 10:22:06,None,A643,KEYSTONE
3,Trenton,Philadelphia,Departed,2018-01-30 11:00:07,None,A643,KEYSTONE


In [11]:
examine_train('2018_02_02_A643')

,from,to,status,time,expected,train_id,line
stop_num,,,,,,,
1,Newark Penn Station,Newark Penn Station,Departed,2018-02-02 09:46:12,None,A643,KEYSTONE
2,Newark Penn Station,Trenton,Departed,2018-02-02 10:23:11,None,A643,KEYSTONE
3,Trenton,Philadelphia,Departed,2018-02-02 11:00:05,None,A643,KEYSTONE


In [10]:
trip_stops[(trip_stops['block_id'] == '3806') & (trip_stops['trip_id'] == 2253)]

,trip_id,expected,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,trip_headsign,direction_id,block_id,shape_id
32317,2253,03:48:00,03:48:00,148,1,0,0,0.0000,9,8,NEW YORK PENN STATION,0,3806,1300
32318,2253,03:54:00,03:54:00,32905,2,0,0,3.6538,9,8,NEW YORK PENN STATION,0,3806,1300
32319,2253,04:00:00,04:00:00,125,3,0,0,9.6248,9,8,NEW YORK PENN STATION,0,3806,1300
32320,2253,04:14:00,04:14:00,103,4,0,0,25.3610,9,8,NEW YORK PENN STATION,0,3806,1300
32321,2253,04:18:00,04:18:00,38,5,0,0,27.7278,9,8,NEW YORK PENN STATION,0,3806,1300
32322,2253,04:23:00,04:23:00,84,6,0,0,30.8036,9,8,NEW YORK PENN STATION,0,3806,1300
32323,2253,04:27:00,04:27:00,83,7,0,0,33.4072,9,8,NEW YORK PENN STATION,0,3806,1300
32324,2253,04:34:00,04:34:00,127,8,0,0,37.2072,9,8,NEW YORK PENN STATION,0,3806,1300
32325,2253,04:38:00,04:38:00,70,9,0,0,39.2657,9,8,NEW YORK PENN STATION,0,3806,1300
32326,2253,04:44:00,04:44:00,41,10,0,0,42.5492,9,8,NEW YORK PENN STATION,0,3806,1300
